<a href="https://colab.research.google.com/github/bpushan/AMPBA_G18/blob/NLP/IIW_Summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IIW dataset is loaded.

In [1]:
!pip install datasets
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# `name` can be one of: IIW-400, DCI_Test, DOCCI_Test, CM_3600, LocNar_Eval
# refer: https://github.com/google/imageinwords/tree/main/datasets
dataset = load_dataset('google/imageinwords', token=None, name="IIW-400", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
dict_IIW = {'image/key': [], 'IIW': []}
dict_IIW['image/key'] = dataset['test']['image/key']
dict_IIW['IIW'] = dataset['test']['IIW']

In [5]:
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter

## Converting the dictionary into a dataframe for the ease of use.

In [6]:
df = df = pd.DataFrame.from_dict(dict_IIW, orient='columns')

## Pre-Process the IIW captions

In [15]:
import re
def preprocess_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

# Apply the preprocessing function to the 'summarized_IIW' column
df['processed_IIW'] = df['IIW'].apply(preprocess_text)

## Performing some analysis of the captions
#### unique word count, Caption with the higest number of words

In [7]:
# Function to tokenize comments and calculate token counts and unique word counts
def analyze_comment(comment):
    tokens = word_tokenize(comment)
    token_count = len(tokens)
    return tokens, token_count

In [16]:
# Apply the function to the 'processed_IIW' column
df['tokens'], df['token_count'] = zip(*df['processed_IIW'].apply(analyze_comment))

In [17]:
# Calculate unique word count across all comments
all_words = [word for tokens in df['tokens'] for word in tokens]
unique_word_count = len(set(all_words))
word_freq = Counter(all_words)

In [18]:
# Find the index of the comment with the highest token count
max_tokens_index = df['token_count'].idxmax()

# Get the comment with the highest token count
comment_with_max_tokens = df.loc[max_tokens_index, 'processed_IIW']
max_token_count = df.loc[max_tokens_index, 'token_count']

In [20]:
# Display the unique word count
print(f"Total number of unique words: {unique_word_count}")

# Display the word frequencies (top 10 for brevity)
print("Top 10 word frequencies:")
for word, freq in word_freq.most_common(10):
    print(f"{word}: {freq}")

# Display the highest number of tokens
print(f"Max number of tokens: {max_token_count}")

Total number of unique words: 6084
Top 10 word frequencies:
the: 7962
a: 3822
of: 2815
and: 2287
is: 1929
in: 1631
with: 1382
on: 1212
to: 1111
are: 827
Max number of tokens: 432


## Since the captions are long, it will be wise to summarize the content and then train for captioning.

In [21]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [22]:
## Let's use the T5-Small model to get the summarized captions to train with.
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [35]:
# Function to summarize text to a specified number of words
def summarize_text(text, max=120, min=40):
    # Tokenize the input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    # Generate summary
    summary_ids = model.generate(inputs, max_length=max, min_length=min, length_penalty=2.0, num_beams=4, early_stopping=True)
    # Decode the generated summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    # return the generated summary
    return summary

#### Test out the summarization function

In [36]:
para = df['processed_IIW'][0]
print("Orginal paragraph: ", para)
sum = summarize_text(para)
print("Summary paragraph: ", sum)

Orginal paragraph:  a closeup outdoor shot shows an echinops bannaticus blue glow globe flower in front of other flowers of that ilk in front of a blue sky and an outoffocus background the flowers spiky extensions are light purpleblue in color with curled tips of dark brown the stem is pale tan and appears to be fuzzy at the bottomleft is a clear focused dark green leaf directed toward the viewer with side leaves going out one to each side but the bloom high above those leaves is outoffocus and dark
Summary paragraph:  a closeup outdoor shot shows an echinops bannaticus blue glow globe flower in front of other flowers of that ilk in front of a blue sky. the flowers spiky extensions are light purpleblue in color with curled tips of dark brown the stem is pale tan and appears to be fuzzy at the bottomleft is a clear focused dark green leaf directed toward the viewer.


In [42]:
# Apply the summarization function to the 'processed_IIW' column
df['summary_IIW'] = df['processed_IIW'].apply(summarize_text)

In [43]:
column_names = df.columns.tolist()
print(column_names)

['image/key', 'IIW', 'tokens', 'token_count', 'processed_IIW', 'summary_IIW']


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
# Create a new DataFrame with only the selected columns
columns_to_export = ['image/key', 'IIW', 'processed_IIW', 'summary_IIW']
df_subset = df[columns_to_export].copy()

In [47]:
# Define the output JSON file path
json_file_path = '/content/drive/MyDrive/NLP/output_json.json'

# Write the DataFrame to a JSON file
df_subset.to_json(json_file_path, orient='records', lines=True)

print(f"Data saved to {json_file_path}")

Data saved to /content/drive/MyDrive/NLP/output_json.json
